In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
class NN:
    def __init__(self, hidden_layers):
        hidden_layers.insert(0,77)
        hidden_layers.append(1)
        self.layers=hidden_layers
        self.L=len(hidden_layers)
        print(self.L)
        
        self.beta={}
        self.gamma={}
        self.W={}
        self.b={}
        
        self.dbeta={}
        self.dgamma={}
        self.dW={}
        self.db={}
        self.initialize_parameters()
        
    def initialize_parameters(self):
        for i in range(1, self.L):
            self.W[i]=tf.Variable(tf.random.normal(shape=(self.layers[i],self.layers[i-1])))
            self.b[i]=tf.Variable(tf.random.normal(shape=(self.layers[i],1)))
            self.beta[i]=tf.Variable(tf.random.normal(shape=(1,1)))
            self.gamma[i]=tf.Variable(tf.random.normal(shape=(1,1)))
            
    def forward_pass(self,A):
        for i in range(1, self.L):
            Z_norm, _ = tf.linalg.normalize(tf.matmul(self.W[i],A)+self.b[i], axis=1)
            Z=self.gamma[i]*Z_norm+self.beta[i]
            A=tf.nn.sigmoid(Z)
        return A
    
    def compute_cost(self,Yhat,Y):
        cost=tf.sqrt(tf.reduce_mean(tf.math.squared_difference(Yhat, Y))) #root mean square error cost function
        return cost
    
    def evaluate_rmse(self, predicted, Y):
        rmse=tf.sqrt(tf.reduce_mean(tf.math.squared_difference(predicted, Y))) #functions are same, because so far I use RMSE as cost func
        return rmse
    
    def update_parameters(self,learning_rate, normalization_lr):
            for i in range(1,self.L):
                self.W[i].assign_sub(learning_rate*self.dW[i])
                self.b[i].assign_sub(learning_rate*self.db[i])
                self.beta[i].assign_sub(normalization_lr*self.dbeta[i])
                self.gamma[i].assign_sub(normalization_lr*self.dgamma[i])
    
    def train_on_batch(self, X_batch, Y_batch, learning_rate, normalization_lr):
        X_batch=tf.convert_to_tensor(X_batch, dtype=tf.float32)
        Y_batch=tf.convert_to_tensor(Y_batch, dtype=tf.float32)
        with tf.GradientTape(persistent=True) as g:
            Yhat=self.forward_pass(X_batch)
            cost=self.compute_cost(Yhat,Y_batch)
        for j in range(1,self.L):
            self.dW[j]=g.gradient(cost, self.W[j])
            self.db[j]=g.gradient(cost, self.b[j])
            self.dbeta[j]=g.gradient(cost, self.beta[j])
            self.dgamma[j]=g.gradient(cost,self.gamma[j])
        del g
        self.update_parameters(learning_rate, normalization_lr)
        return cost.numpy()
        
    def train(self, X, Y, epochs, batch_size, learning_rate, decay_rate, normalization_lr):
        steps_per_epoch_non_floored=X.shape[1]/batch_size
        steps_per_epoch=int(steps_per_epoch_non_floored)
        
        for e in range(epochs):
            epoch_loss=0
            learning_rate=learning_rate/(1+decay_rate*e)
            for i in range(steps_per_epoch):
                X_batch=X[:,i*batch_size:(i+1)*batch_size]
                Y_batch=Y[:,i*batch_size:(i+1)*batch_size]
                batch_loss=self.train_on_batch(X_batch, Y_batch, learning_rate, normalization_lr)
                epoch_loss+=batch_loss
            
            if steps_per_epoch<steps_per_epoch_non_floored:
                X_last_batch=X[:,(i+1)*batch_size:]
                Y_last_batch=Y[:,(i+1)*batch_size:]
                batch_loss=self.train_on_batch(X_last_batch, Y_last_batch, learning_rate, normalization_lr)
                epoch_loss+=batch_loss
            
            if(e%100==0):
                print("Loss on epoch {} is: ".format(e),epoch_loss/steps_per_epoch)
    
    
    def predict(self, X):
        X=tf.convert_to_tensor(X, dtype=tf.float32)
        prediction=self.forward_pass(X)
        return prediction
    
    def upload_parameters(self,W,b,beta,gamma):
        self.W=W
        self.b=b
        self.beta=beta
        self.gamma=gamma
            
    
                             

In [ ]:
"""
#первый датасет Доры
X_train = pd.read_csv('nonorm_negignore_knn3/x_knn_train_negative.csv', header=0).values[:, 1:].T
Y_train = pd.read_csv('nonorm_negignore_knn3/y_knn_train_negative.csv', header=0).values[:, 1:].T
X_test = pd.read_csv('nonorm_negignore_knn3/x_knn_test_negative.csv', header=0).values[:, 1:].T
Y_test = pd.read_csv('nonorm_negignore_knn3/y_knn_test_negative.csv', header=0).values[:, 1:].T

#второй датасет Доры
X_train = pd.read_csv('norm_negignore_knn3/x_knn_train_negative_normalized.csv', header=0).values[:, 1:].T
Y_train = pd.read_csv('norm_negignore_knn3/y_knn_train_negative_normalized.csv', header=0).values[:, 1:].T
X_test = pd.read_csv('norm_negignore_knn3/x_knn_test_negative_normalized.csv', header=0).values[:, 1:].T
Y_test = pd.read_csv('norm_negignore_knn3/y_knn_test_negative_normalized.csv', header=0).values[:, 1:].T

#третий датасет Доры
X_train = pd.read_csv('norm_negignore_knn3+mean/x_train_knn+mean.csv', header=0).values[:, 1:].T
Y_train = pd.read_csv('norm_negignore_knn3+mean/y_train_knn+mean.csv', header=0).values[:, 1:].T
X_test = pd.read_csv('norm_negignore_knn3+mean/x_test_knn+mean.csv', header=0).values[:, 1:].T
Y_test = pd.read_csv('norm_negignore_knn3+mean/y_test_knn+mean.csv', header=0).values[:, 1:].T
"""
#интерполяция + knn
X_train = pd.read_csv('datasets/norm_negignore_interpol_knn3/x_train_knn+interpol.csv', header=0).values[:, 1:].T
Y_train = pd.read_csv('datasets/norm_negignore_interpol_knn3/y_train_knn+interpol.csv', header=0).values[:, 1:].T
X_test = pd.read_csv('datasets/norm_negignore_interpol_knn3/x_test_knn+interpol.csv', header=0).values[:, 1:].T
Y_test = pd.read_csv('datasets/norm_negignore_interpol_knn3/y_test_knn+interpol.csv', header=0).values[:, 1:].T

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
#параметры модели

hidden_layers=[70,50] #кол-во элементов это кол-во hidden layers, значение элемента это кол-во neurons в этом layer
learning_rate=0.01
decay_rate=0

normalization_lr=0.001
batch_size=512

epochs=15000

In [ ]:
network=NN(hidden_layers)

In [ ]:
network.train(X_train, Y_train, epochs, batch_size, learning_rate, decay_rate, normalization_lr)

In [ ]:
predictions_train=network.predict(X_train)
error_train=network.evaluate_rmse(predictions_train,Y_train)

predictions_test=network.predict(X_test)
error_test=network.evaluate_rmse(predictions_test,Y_test)

print("Average error on train set: ", error_train.numpy())
print("Average error on test set: ", error_test.numpy())

In [ ]:
a=tf.Variable(tf.random.normal(shape=(3,5)))
a=tf.convert_to_tensor(a, dtype=tf.float32)
print(a)
a, norm=tf.linalg.normalize(a, axis=1)
print(a)
print(norm)